## WKT to Segmentation Mask

This notebook illustrates an example to convert segmentation WKTs into grayscale image mask.

This example uses:  
- `PIL` library for image I/O and drawing mask
- `shapely` for pre-processing WKTs and generating polygons

If you recieved your WKTs from Centaur, they will be formatted with relative coordinates.
Make sure to convert them to absolute coordinates first. Check the (@TODO: Link recipe)
recipe to do the conversion.

This script does not handle shapes with holes.

In [ ]:
# imports
import json
import numpy as np

from PIL import Image, ImageDraw

import shapely.wkt
import shapely.geometry

In [ ]:
# inputs paths
JSON_WKT_PATH = './example/input/example_mask_wkt.json'
BASE_IMAGE_FILE_PATH = './example/input/example_base.png'
# output path
MASK_IMAGE_FILE_PATH = './example/output/example_mask.png'

### segmentation wkts to grayscale mask

In [ ]:
# load wkts from json
with open(JSON_WKT_PATH) as f:
    list_of_wkt_masks = json.load(f)
list_of_wkt_masks

In [ ]:
# load base image
base_image = Image.open(BASE_IMAGE_FILE_PATH).convert('L')
# prepare blank canvas
canvas = np.zeros((int(base_image.height), int(base_image.width)))

# draw mask
mask_image = Image.fromarray(np.uint8(canvas))
for wkt_mask in list_of_wkt_masks:
    mask_polygon = shapely.wkt.loads(wkt_mask["wkt"])
    if mask_polygon.is_empty:
            continue
    mask_poly_coords = list(mask_polygon.exterior.coords)
    ImageDraw.Draw(mask_image).polygon(mask_poly_coords, fill = wkt_mask['intensity'])
    
# save mask
mask_image.save(MASK_IMAGE_FILE_PATH)

### visualize generated mask on base image

In [ ]:
from PIL import ImageFont
from matplotlib import font_manager
font = font_manager.FontProperties(family='sans-serif', weight='bold')
font = ImageFont.truetype(font_manager.findfont(font), 48)

# overlay mask
composite_image = Image.blend(mask_image, base_image, alpha=0.6)

# show overlay
ImageDraw.Draw(base_image := base_image.convert("RGBA")).text((100, 100), "Base Image", (255,255,255), font=font)
ImageDraw.Draw(mask_image := mask_image.convert("RGBA")).text((100, 100), "Mask Image", (255,255,255), font=font)
ImageDraw.Draw(composite_image := composite_image.convert("RGBA")).text((100, 100), "Mask Overlay", (255,255,255), font=font)
overlay_strip = Image.fromarray(np.hstack((np.array(base_image), 
                                           np.array(mask_image), 
                                           np.array(composite_image))))
overlay_strip